<img src="https://github.com/OasisLMF/Workshop2019/raw/master/images/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250" align="left"/>
<br><br><br>

Exercise 2: Model Installation 
===============================



This version of the OasisPlatform differs from the existing oasis releases, model workers build from the existing oasis versions, such as [1.26.3](https://github.com/OasisLMF/OasisPlatform/releases/tag/1.26.3), [1.23.13](https://github.com/OasisLMF/OasisPlatform/releases/tag/1.23.13-lts) and [1.15.26](https://github.com/OasisLMF/OasisPlatform/releases/tag/1.15.26-lts) are not directly compatible with Enterprise Platform components which are tag with `2.x.x` version numbers. To port models from the `1.x.x` platform to `2.x.x`, the base worker needs to be build from [Dockerfile.model_worker](https://raw.githubusercontent.com/OasisLMF/OasisPlatform/platform-2.0-develop/Dockerfile.model_worker) on the [platform-2.0-develop](https://github.com/OasisLMF/OasisPlatform/tree/platform-2.0-develop) branch in the OasisPlatform Github repository.  

Both the example models, `PiWind` and `CHAZ`, in this workshop are data driven. Since all of the components are 'Vanilla oasis' and don't require custom supplier components or code, the installation process is straightforward. Part of the platform installation script pushes a version `2.x.x` Oasis worker to the [Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-intro) two run either of the data driven models we just need to make sure that this image is referenced in the deployment yaml configuration (covered in section 2.2.3)    

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/azure_container_reg.png?raw=true" alt="Azure Portal Home" width="800" align="center" style="float"/><br>



### Migrating models (rule of thumb) 
* **Data driven models (low effort):**  change the default `model_worker` docker image. 
* **Custom components (medium effort):**  requires rebuilding a new supplier image based on `2.x.x` worker
* **Complex models (case by case):**  This depends on the complexity of the custom components and how practical it is to parallelize a custom GUL across multiple worker nodes.      


### Pipeline Automation for Model deployment 
The platform and model deployment scripts were written for [Azure DevOps pipeline](https://learn.microsoft.com/en-us/azure/devops/pipelines/get-started/what-is-azure-pipelines?view=azure-devops). While we won't be covering Pipelines in this workshop, there is [pipeline how-to guide](https://github.com/OasisLMF/OasisAzureDeployment#22-prepare-devops-pipeline) in the OasisAzureDeployment Readme file.

## 2.1 Install PiWind using [OasisAzureDeployment](https://github.com/OasisLMF/OasisAzureDeployment)


### 2.1.1 Git clone 

Start an azure cloud shell session and Git clone the following GitHub repositories. The terminal icons followed by `some text` are command line which should work 'as is' by copying into https://shell.azure.com/ 

1. [OasisAzureDeployment](https://github.com/OasisLMF/OasisAzureDeployment) 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `git clone https://github.com/OasisLMF/OasisAzureDeployment.git -b develop`

2. [OasisPlatform](https://github.com/OasisLMF/OasisPlatform) 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `git clone --depth 1 https://github.com/OasisLMF/OasisPlatform.git -b platform-2.0-develop ~/OasisPlatform`
 

3. [OasisPiWind](https://github.com/OasisLMF/OasisPiWind) 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `git clone --depth 1 https://github.com/OasisLMF/OasisPiWind.git -b develop ~/OasisPiWind`


### 2.1.2 Update the `settings/settings.sh` file
There are shell variables that need to be set for the **OasisAzureDeployment** scripts to work, paste and execute the folloing commands into the shell. 
While these arn't strictly needed for the PiWind deployment command to work, the script will throw an error without these set correctly;. 


1. Set **DNS_LABEL_NAME** 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `sed -i "s|^DNS_LABEL_NAME.*|DNS_LABEL_NAME=oasis-workshop-$(echo $USER | tr -dc '0-9')|" ~/OasisAzureDeployment/settings/settings.sh`

2. Set **RESOURCE_GROUP** 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `sed -i "s|^RESOURCE_GROUP.*|RESOURCE_GROUP=oasis-workshop-$(echo $USER | tr -dc '0-9')|" ~/OasisAzureDeployment/settings/settings.sh`

3. Set **LETSENCRYPT_EMAIL** 
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `sed -i "s|^LETSENCRYPT_EMAIL.*|LETSENCRYPT_EMAIL=workshop$(echo $USER | tr -dc '0-9')@oasislmfenterprise.onmicrosoft.com|" ~/OasisAzureDeployment/settings/settings.sh`


### 2.1.3 Run the PiWind installation command 

1. Double check the **deploy.sh** script works with the added variables by running the help command. If set correctly, you should see the following list of available commands 

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `cd ~/OasisAzureDeployment && ./deploy.sh --help`



```
Usage: ./deploy.sh <command>

Available commands:
  resource-group Create the resource group in Azure to be used for the Oasis platform
    ...
```


2. Run the Deploy PiWind command, this will push files to the Azure file share mounted to worker pods and create a [Kubernetes deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) for PiWind. Part of the deployment process includes testing, where the example portfolios are executed to check that the deployment was successful.  

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `./deploy.sh piwind`

### 2.1.4 Fetch Data from the API to check installation

The following python fetches data from https://oasis-workshop-{id}.northcentralus.cloudapp.azure.com/api/ 
and presents the json returns as tables. 

> **Alternative method:**  accessing OasisUI directly from https://oasis-workshop-{id}.northcentralus.cloudapp.azure.com/ (replace `{id}` with your workshop number)

In [ ]:
# Edit this value to match your workshop ID, if your username is `workshop6@oasislmfenterprise.onmicrosoft.com` set   'WORKSHOP_ID=6'
WORKSHOP_ID=

# Install of workshop package 
!pip install oasis-workshop==1.0.0
    
# Clear cell output
from IPython.display import clear_output
clear_output(wait=False) 

In [ ]:
# Python imports and setup 
import time 
import json    
import tabulate

from oasis_workshop.client import APIClient
from oasis_workshop.funcs import tabulate_endpoint, tabulate_json
from IPython.display import display, clear_output, HTML, Markdown 
    
# Start API client session     
oasis_server = APIClient(
    api_url=f'https://oasis-workshop-{WORKSHOP_ID}.northcentralus.cloudapp.azure.com/api/',
    username='admin', 
    password='password'
)

In [ ]:
models = tabulate_endpoint(oasis_server.models, ['id', 'supplier_id', 'model_id', 'version_id'])
portfolios = tabulate_endpoint(oasis_server.portfolios, ['id', 'name', 'location_file', 'accounts_file', 'reinsurance_info_file', 'reinsurance_scope_file'])
analyses = tabulate_endpoint(oasis_server.analyses, ['id', 'name', 'model', 'portfolio', 'status', 'input_file', 'output_file', 'run_log_file'])

display(Markdown(f'### Installed Models'))
display(HTML(models))

display(Markdown(f'### Portfolios Table'))
display(HTML(portfolios))

display(Markdown(f'### Analyses Table'))
display(HTML(analyses))

## 2.2 Install and configure CHAZ (Columbia - Tropical Cyclone)

### 2.2.1 Download a local copy of the CHAZ model data to Azure Cloud Shell

There is a copy of the CHAZ model data stored in [Azure Blob storage](https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blobs-overview), in this exercise the idea is to deploy this to the cluster, via a more manual process than the piwind installation. 


<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/model_blob_storage.png?raw=true" alt="Azure Portal Home" width="600" align="center" style="float"/><br>


* **Step 1** - Open Azure cloud shell and login with the [az utility](https://learn.microsoft.com/en-us/cli/azure/what-is-azure-cli) this is a command line tool to interact with Azure resources.

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `az login --identity`

```
  {
    "environmentName": "AzureCloud",
    "homeTenantId": "812eb12f-c995-45ef-bfea-eac7aeca0755",
    "id": "da90aaa5-0e0f-4362-bb41-422887865d0f",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Oasis-Infrastructure",
    "state": "Enabled",
    "tenantId": "812eb12f-c995-45ef-bfea-eac7aeca0755",
    "user": {
      "cloudShellID": true,
      "name": "workshop1@oasislmfenterprise.onmicrosoft.com",
      "type": "user"
    }
```


* **Step 2** - Issue a [SAS token](https://learn.microsoft.com/en-us/azure/storage/common/storage-sas-overview) which grants access to download the CHAZ model data

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `CONTAINER_SAS_TOKEN=$(az storage container generate-sas --account-name oasismodelstore --name columbia-chaz  --permissions acdlrw --expiry $(date -u -d "300 minutes" '+%Y-%m-%dT%H:%MZ') | tr -d '"')
`

Echo out the `CONTAINER_SAS_TOKEN` shell variable to double check the SAS token was created 
```
workshop1 [ ~ ]$ echo $CONTAINER_SAS_TOKEN
se=2022-09-23T14%3A26Z&sp=racwdl&sv=2021-06-08&sr=c&sig=3c5F7iyp%2BCVy%2BK1hbN8kAQOl7dRNA4mAMsYE0clOE9k%3D
```



* **Step 3** - Use the [azcopy](https://learn.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) tool to download a localy copy of the CHAZ data. 

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `azcopy copy "https://oasismodelstore.blob.core.windows.net/columbia-chaz/*?$CONTAINER_SAS_TOKEN" "~/chaz-data" --recursive=true --overwrite=ifSourceNewer`


**Example Output**
```
true --overwrite=ifSourceNewer
INFO: Scanning...
INFO: Any empty folders will not be processed, because source and/or destination doesn't have full folder support
INFO: azcopy: A newer version 10.16.0 is available to download


Job 0675edeb-80ba-7443-5fd1-1177122a44c9 has started
Log file is located at: /home/workshop1/.azcopy/0675edeb-80ba-7443-5fd1-1177122a44c9.log

100.0 %, 14 Done, 0 Failed, 0 Pending, 0 Skipped, 14 Total,                                


Job 0675edeb-80ba-7443-5fd1-1177122a44c9 summary
Elapsed Time (Minutes): 0.0667
Number of File Transfers: 14
Number of Folder Property Transfers: 0
Total Number of Transfers: 14
Number of Transfers Completed: 14
Number of Transfers Failed: 0
Number of Transfers Skipped: 0
TotalBytesTransferred: 125452308
Final Job Status: Completed
```

### 2.2.2 Edit the meta-data files and upload to cluster 

You should now have a local copy of the CHAZ model data downloaded to `~/chaz-data`, if not ask for assisstance! 

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/local_model_data.png?raw=true" alt="Azure shell" width="800" align="center" style="float"/><br>

Next, using either [nano](https://www.nano-editor.org/dist/latest/cheatsheet.html) or [vim](https://vim.rtorr.com/) make the following edits to the `chunking_configuration.json` and `scaling_configuration.json`  files. This needs to happen before uploading these to the cluster file share. 


**Editing - chunking_configuration.json** - Update the default chunking config values, this affects how many pieces (or chunks) an analysis is broken up into (these options are covered in the next notebook)

```
"lookup_strategy": "DYNAMIC_CHUNKS",
"loss_strategy": "DYNAMIC_CHUNKS",
"dynamic_locations_per_lookup": 11000,
"dynamic_events_per_analysis": 4800,
```

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure Portal Home" width="28" align="left"/>  `nano ~/chaz-data/chunking_configuration.json`

> **Hint:** the shortcut `Ctrl-X` saves changes to the file and exits nano



<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/nano_chunking_config.png?raw=true" alt="Azure shell" width="400" align="center" style="float"/><br>


**Editing - scaling_configuration.json** -Update the auto-scaling defaults, this changes how many workers are created to process each of the the job chunks in parallel (these options are covered in the next notebook) 

```
 "worker_count_fixed": 4,
```
> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure shell" width="28" align="left"/>  `nano ~/chaz-data/scaling_configuration.json`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/nano_scaling_config.png?raw=true" alt="Azure shell" width="400" align="center" style="float"/><br>

### 2.2.3 Upload the model data 

Using the `upload_model_data.sh` from **OasisAzureDeployment** upload the modified model files to the cluster fileshare. This is a file system which gets mounted to the worker containers on startup. 

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure shell" width="28" align="left"/>  `~/OasisAzureDeployment/scripts/upload_model_data.sh -C ~/chaz-data/ columbia/chaz/1 $(ls ~/chaz-data/)`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/upload_chaz.png?raw=true" alt="Azure shell" width="400" align="center" style="float"/><br>


If the command was successful you should be able to see the files in the clusters file share, located under `models/columbia/chaz/1`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/chaz_model_fileshare.png?raw=true" alt="Azure shell" width="600" align="center" style="float"/><br>



### 2.2.3 Create a deployment YML and run the model deployment command 

Each model requires its own [kubernetes deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), to set this up we'll need to define that deployment with a YAML configuration file. The configuration for PiWind looks like this [oasislmf-piwind-1.yaml](https://raw.githubusercontent.com/OasisLMF/OasisAzureDeployment/master/settings/helm/workers/oasislmf-piwind-1.yaml) to adapt this file for CHAZ, all that needs to change is the top section specifying the model's name. 

**FROM:**
```
workers:
  oasislmf_piwind_1: # A name that is unique among all workers
    supplierId: OasisLMF  # Must be identical to supplier in the model data file share
    modelId: PiWind       # Must be identical to name in the model data file share
    modelVersionId: "1"   # Must be identical to version in the model data file share
     ... 
```

**TO:**
```
workers:
  columbia_chaz_1:                        # A name that is unique among all workers
    supplierId: columbia                  # Must be identical to supplier in the model data file share
    modelId: chaz                         # Must be identical to name in the model data file share
    modelVersionId: "1"     
      ...
```

* To skip editing this file you can use the shortcut below to download an example directly. 


> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure shell" width="28" align="left"/>  `wget https://raw.githubusercontent.com/OasisLMF/Workshop2022/main/examples/columbia-chaz-1.yaml -O ~/columbia-chaz-1.yaml`



* Once this file is ready move it to the worker deployment subfolder of **OasisAzureDeployment**

> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure shell" width="28" align="left"/>  `mv columbia-chaz-1.yaml ~/OasisAzureDeployment/settings/helm/workers/`



* Running the deploy models command will automaticlly pickup and deploy all model YAML files from the [settings/helm/workers/](https://github.com/OasisLMF/OasisAzureDeployment/tree/master/settings/helm/workers) directory 


> <img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/terminal_icon.png?raw=true" alt="Azure shell" width="28" align="left"/>  `~/OasisAzureDeployment/deploy.sh models`


**Example output**
```
Deploying models...
Updating kubectl cluster
Helm chart upgrade...
Release "models" has been upgraded. Happy Helming!
NAME: models
LAST DEPLOYED: Fri Sep 23 11:37:44 2022
NAMESPACE: default
STATUS: deployed
REVISION: 3
TEST SUITE: None
NOTES:
Model(s) registered and worker(s) deployed.
Helm finished
Waiting for models to be registered: 
chaz.../home/workshop1/OasisAzureDeployment/deploy.sh: line 189: kill: (819) - No such process

PiWind...
Models deployed
```


### 2.2.4 check the model registration worked 

If the above commands worked, you should see two model deployments via kubectl. 
* `kubectl get deployments | grep ^worker` 


<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/model_deployed.png?raw=true" alt="Azure shell" width="600" align="center" style="float"/><br>

* `kubectl describe deployment worker-columbia-chaz-1`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/model_deploy_details.png?raw=true" alt="Azure shell" width="600" align="center" style="float"/><br>




If something fails you will see that in the **model-registration** pod execution, if for example, the `model_settings.json` file is invalid it would show up as an error. 



* `kubectl get pods | grep model-registration`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/model_reg.png?raw=true" alt="Azure shell" width="600" align="center" style="float"/><br>

* `kubectl logs model-registration-ac0tij-5ht5n`

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/model_reg_details.png?raw=true" alt="Azure shell" width="600" align="center" style="float"/><br>


In [ ]:
models = tabulate_endpoint(oasis_server.models, ['id', 'supplier_id', 'model_id', 'version_id'])
display(Markdown(f'### Installed Models'))
display(HTML(models))